In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
source.content
soup = BeautifulSoup(source.content, 'html.parser')
stat_table =soup.find_all('table', class_ = 'wikitable' )
stat_table = stat_table[0]
with open ('canada_postcode.txt', 'w') as r:
    for row in stat_table.find_all('tr'):
        for cell in row.find_all('td'):
            r.write('  ')
            r.write(cell.text)       
headers = ['Postal Code', 'Borough', 'Neighborhood']
import pandas as pd
df = pd.read_csv('canada_postcode.txt', sep= "  ", header = None, names = headers)
df = df[df.Borough != 'Not assigned']
df = df.reset_index(drop=True)
df = (df.groupby(['Postal Code','Borough'],as_index = False, sort = False)['Neighborhood'].aggregate(', '.join))
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)
path='https://cocl.us/Geospatial_data'
df2 = pd.read_csv(path)
df3 = pd.merge(df, df2, on='Postal Code')

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [2]:
!pip install folium

In [3]:
from geopy.geocoders import Nominatim
import requests
import folium
import json
import pandas.io.json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

In [6]:
address = 'Toronto'

geolocator = Nominatim(user_agent = 'ca_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location = [latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color ='#3186cc',
        fill_opacity = 0.7,
        parse_html=False).add_to(map_toronto)
map_toronto